<a href="https://colab.research.google.com/github/PraeJi/Python-Practice/blob/master/3_05_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Loading the Cats vs Dogs dataset

In [ ]:
# Load the "cats vs dogs" dataset using tensorflow_datasets
# Split the dataset into 3 subsets:
#   70% for training,
#   15% for validation,
#   15% for testing
d_train, d_valid, d_test = tfds.load(
    "cats_vs_dogs",
    split=["train[:70%]", "train[70%:85%]", "train[85%:100%]"],
    as_supervised=True,
)

# Display the number of examples in each subset
print("n_train = %d" % tf.data.experimental.cardinality(d_train))
print("n_valid = %d" % tf.data.experimental.cardinality(d_valid))
print("n_test = %d" % tf.data.experimental.cardinality(d_test))

In [ ]:
# Preprocess
#
# Resize each image into 64x64
size = (64, 64)
d_train = d_train.map(lambda x, y: (tf.image.resize(x, size), y))
d_valid = d_valid.map(lambda x, y: (tf.image.resize(x, size), y))
d_test = d_test.map(lambda x, y: (tf.image.resize(x, size), y))

# Normalize the color values
d_train = d_train.map(lambda x, y: (x / 255, y))
d_valid = d_valid.map(lambda x, y: (x / 255, y))
d_test = d_test.map(lambda x, y: (x / 255, y))

# Set up batches
batch_size = 32
d_train = d_train.cache().batch(batch_size).prefetch(buffer_size=10)
d_test = d_test.cache().batch(batch_size).prefetch(buffer_size=10)
d_valid = d_valid.cache().batch(batch_size).prefetch(buffer_size=10)

# Loading the VGG16 model

In [ ]:
# Load the VGG16 model and use it as the base model
# Use the weights trained by the imagenent dataset
base_model = tf.keras.applications.VGG16(weights='imagenet',
                                         input_shape=(64, 64, 3),
                                         include_top=False)

# Defining the classification part

In [ ]:
# Freeze the weight of VGG16
base_model.trainable = False

# Set up the classification part of the model
inputs = tf.keras.Input(shape=(64, 64, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
x = tf.keras.layers.Dense(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

# Training the classification part

In [ ]:
# Train the model
model.fit(d_train,
          epochs=20,
          batch_size=batch_size,
          shuffle=True, validation_data=d_valid)

# Evaluate the model using the test set
model.evaluate(d_test, verbose=0)

r = model.evaluate(d_test, verbose=0)
print(f"Test accuracy 1 = {r[1]:.4f}")

# Finetuning the model

In [ ]:
# Fine tune the model
base_model.trainable = True
model.summary()

# The learning rate is set to a very small value
model.compile(optimizer=tf.keras.optimizers.Adam(1e-6),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Training the entire model
model.fit(d_train, epochs=10, batch_size=batch_size, shuffle=True,
          validation_data=d_valid)

# Evaluate the model using the test set
r = model.evaluate(d_test, verbose=0)
print(f"Test accuracy 2 = {r[1]:.4f}")

In [ ]:
X_test1, y_test1 = list(d_test)[0]

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_test1[0])
plt.show()

In [ ]:
y_pred = model.predict(X_test1[0:2])
tf.print(y_pred)
tf.print(y_test1[0:2])